In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from pydub import AudioSegment
import os
import math
from scipy.interpolate import interp1d
import pandas as pd
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
import csv
import japanize_matplotlib
from data import *

     ボーカル  メロディ  ドラム  全身  ボーカル.1  メロディ.1  ドラム.1  右手  右足  左手  左足  右肘  右膝  左肘  \
0     0.0   1.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   
1     0.0   1.0  0.0   1     1.0     1.0    0.0   1   0   1   1   0   0   0   
2     0.0   1.0  0.0   0     1.0     0.0    0.0   0   0   0   0   0   0   0   
3     0.0   1.0  1.0   0     1.0     0.0    1.0   0   0   0   0   0   0   0   
4     0.0   0.0  0.0   0     0.0     0.0    1.0   0   0   0   0   0   0   0   
..    ...   ...  ...  ..     ...     ...    ...  ..  ..  ..  ..  ..  ..  ..   
187   0.0   0.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   
188   0.0   0.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   
189   0.0   0.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   
190   0.0   0.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   
191   0.0   0.0  0.0   0     0.0     0.0    0.0   0   0   0   0   0   0   0   

     左膝  
0     1  
1     1  
2     0  
3     0  
4

In [25]:
music_count_length = 96  # UWW: 96  toshi: 90
music_half_count_length = music_count_length * 2
music_quarter_count_length = music_count_length * 4
accel_start_timing = 606  # UWW: 606  toshi: 641
kinect_start_timing = 250   # UWW: 250   toshi: 556
offset=10 # UWW: 10  toshi: 140
duration=65  # UWW: 65  toshi: 60
tempo = 89.10290948
print(tempo)
half_count = float(30/tempo)
quarter_count = float(15/tempo)
half_count_list = []
quarter_count_list = []
for i in range(music_half_count_length):
    half_count_list.append(i * half_count)
    
for l in range(music_quarter_count_length):
    quarter_count_list.append(l * quarter_count)

89.10290948


In [5]:
def part_to_figure(vocal, melody, drum):
    data_length = len(vocal)
    figure_time_list = []
    for i in range(data_length):
        figure_time_list.append(i * 16 * 12 *quarter_count / 100)
    x1 = figure_time_list
    y1 = vocal
    y2 = melody
    y3 = drum


    # Figureの初期化
    fig = plt.figure(figsize=(12, 10)) #...1

    #図の形式定義
    plt.rcParams['font.family'] ='Yu Gothic'#使用するフォント
    plt.rcParams['xtick.direction'] = 'in'#x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
    plt.rcParams['ytick.direction'] = 'in'#y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
    plt.rcParams['xtick.major.width'] = 1.0#x軸主目盛り線の線幅
    plt.rcParams['ytick.major.width'] = 1.0#y軸主目盛り線の線幅
    plt.rcParams['font.size'] = 19 #フォントの大きさ
    plt.rcParams['axes.linewidth'] = 1.0# 軸の線幅edge linewidth。囲みの太さ
    
    label_l = ['', 'systemOnly', 'handmadeOnly', '(0,0)', '(1,1)']

    ax = fig.add_subplot(3, 1, 1)#(行，列，領域番号)
    ax.plot(x1, y1,c="blue",label='Estimated Part',linewidth=0.8)
    ax.set_xlabel('Time[s]',fontsize=22)
#     ax.set_ylabel('acc')
#     ax.set_xlim((0,14000))
    ax.set_ylim((0, 3.5))
    ax.set_title("vocal",fontsize=26)
    # ax.set_title("tf-idf method",fontsize=26)
    ax.yaxis.set_major_locator(mpl.ticker.IndexLocator(1, -1))
    ax.set_yticklabels(label_l, ha='right',fontsize=22)
    # ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize=19)

    ax = fig.add_subplot(3, 1, 2)#(行，列，領域番号)
    ax.plot(x1, y2,c="green",label='Estimated Part',linewidth=0.8)
    ax.set_xlabel('Time[s]',fontsize=22)
#     ax.set_ylabel('acc')
#     ax.set_xlim((0,14000))
    ax.set_ylim((0, 3.5))
    ax.set_title("melody",fontsize=26)
    # ax.set_title("tf-idf method",fontsize=26)
    ax.yaxis.set_major_locator(mpl.ticker.IndexLocator(1, -1))
    ax.set_yticklabels(label_l, ha='right',fontsize=22)
    # ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize=19)

    ax = fig.add_subplot(3, 1, 3)#(行，列，領域番号)
    ax.plot(x1, y3,c="red",label='Estimated Part',linewidth=0.8)
    ax.set_xlabel('Time[s]',fontsize=22)
#     ax.set_ylabel('acc')
#     ax.set_xlim((0,14000))
    ax.set_ylim((0, 3.5))
    ax.set_title("drum",fontsize=26)
    # ax.set_title("tf-idf method",fontsize=26)
    ax.yaxis.set_major_locator(mpl.ticker.IndexLocator(1, -1))
    ax.set_yticklabels(label_l, ha='right',fontsize=22)
    # ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize=19)
    

    plt.tight_layout()#グラフが重ならず，設定した図のサイズ内に収まる。
    plt.show()
#     files.download(figname)

In [6]:
def single_to_figure(lists):
    data_length = len(lists)
    figure_time_list = []
    for i in range(data_length):
        figure_time_list.append(i * 16 * 12 *quarter_count / 100)
    x1 = figure_time_list
    y1 = lists


    # Figureの初期化
    fig = plt.figure(figsize=(12, 5)) #...1

    #図の形式定義
    plt.rcParams['font.family'] ='Yu Gothic'#使用するフォント
    plt.rcParams['xtick.direction'] = 'in'#x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
    plt.rcParams['ytick.direction'] = 'in'#y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
    plt.rcParams['xtick.major.width'] = 1.0#x軸主目盛り線の線幅
    plt.rcParams['ytick.major.width'] = 1.0#y軸主目盛り線の線幅
    plt.rcParams['font.size'] = 19 #フォントの大きさ
    plt.rcParams['axes.linewidth'] = 1.0# 軸の線幅edge linewidth。囲みの太さ
    
    label_l = ['systemOnly', 'handmadeOnly', '(0,0)', '(1,1)']

    ax = fig.add_subplot(1, 1, 1)#(行，列，領域番号)
    ax.plot(x1, y1,c="blue",label='Estimated Part',linewidth=0.8)
    ax.set_xlabel('Time[s]',fontsize=22)
#     ax.set_ylabel('acc')
#     ax.set_xlim((0,14000))
    ax.set_ylim((0, 3.5))
    ax.set_title("body",fontsize=26)
    # ax.set_title("tf-idf method",fontsize=26)
    ax.yaxis.set_major_locator(mpl.ticker.IndexLocator(1, -1))
    ax.set_yticklabels(label_l, ha='right',fontsize=22)
    # ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize=19)



    plt.tight_layout()#グラフが重ならず，設定した図のサイズ内に収まる。
    plt.show()
#     files.download(figname)

In [7]:
def CreateSimilarlyList(moveList, partList):
    result = []
    for move, part in zip(moveList, partList):
        if move == part == 1:
            result.append(1)
        else: result.append(0)
    return result

def CompareSimilarlyListWithHandmade(similaryList, handmadeList):
    result = []
    ok_sum = 0
    miss_num = 0
    for similarly, handmade in zip(similaryList, handmadeList):
        if similarly == handmade:
            ok_sum += 1
            if similarly == 1:
                result.append(3)
            else: result.append(2)
        else:
            miss_num += 1
            if handmade == 1:
                result.append(1)
            else: result.append(0)
    return result, (ok_sum/(ok_sum + miss_num))*100

def partToBody(partsList, threshold):
    result = []
    for i in range(len(partsList[0])):
        sum_part_timing = 0
        for j in range(len(partsList)):
            sum_part_timing += partsList[j][i]
        if sum_part_timing >= threshold:
            result.append(1)
        else: result.append(0)
    return result


In [9]:
HRBase_HM_df = pd.read_csv('data/HRBASE_4.csv', usecols=[i for i in range(2,17)], header=1, encoding="utf-8")
# print(HRBase_HM_df)
similarly_vocal = HRBase_HM_df['ボーカル']
similarly_melody = HRBase_HM_df['メロディ']
similarly_drum = HRBase_HM_df['ドラム']
body = HRBase_HM_df['全身']
vocal = HRBase_HM_df['ボーカル.1']
melody = HRBase_HM_df['メロディ.1']
drum = HRBase_HM_df['ドラム.1']
rightHand = HRBase_HM_df['右手']
rightFoot = HRBase_HM_df['右足']
rightElbow = HRBase_HM_df['右肘']
rightKnee = HRBase_HM_df['右膝']
leftHand = HRBase_HM_df['左手']
leftFoot = HRBase_HM_df['左足']
leftElbow = HRBase_HM_df['左肘']
leftKnee = HRBase_HM_df['左膝']

partsList = [rightHand, rightFoot, rightElbow, rightKnee, leftHand, leftFoot, leftElbow, leftKnee]

vocal_result = CreateSimilarlyList(body, vocal)
melody_result = CreateSimilarlyList(body, melody)
drum_result = CreateSimilarlyList(body, drum)

vocal_accuracy_graph, vocal_accuracy_num = CompareSimilarlyListWithHandmade(vocal_result, similarly_vocal)
melody_accuracy_graph, melody_accuracy_num = CompareSimilarlyListWithHandmade(melody_result, similarly_melody)
drum_accuracy_graph, drum_accuracy_num = CompareSimilarlyListWithHandmade(drum_result, similarly_drum)

print('vocal accyracy: ', vocal_accuracy_num)
print('melody accyracy: ', melody_accuracy_num)
print('drum accyracy: ', drum_accuracy_num)

part_to_figure(vocal_accuracy_graph, melody_accuracy_graph, drum_accuracy_graph)


for i in range(1, 9):
    similarlyBody = partToBody(partsList, i)
    body_accuracy_graph, body_accuracy_num = CompareSimilarlyListWithHandmade(similarlyBody, body)
    print(body_accuracy_graph)

    print('精度: ', body_accuracy_num, 'threshold: ', i)
    single_to_figure(body_accuracy_graph)


vocal accyracy:  87.5
melody accyracy:  86.45833333333334
drum accyracy:  95.83333333333334


NameError: name 'quarter_count' is not defined

# README

## パートと数値の関係
0: None
1: Drum
2: Melody
3: Vocal

## 目視データと出力データの比較の際の数値
0: 出力データのみ発生
1: 目視データのみ発生
2: 両方重複タイミング無し
3: 両方重複タイミングあり